# Lab 02.a - Week 3
## Early Fire Detection with Multi-Layer Perceptron

## **Challenge 01**


First, let's load the saved NumPy array data from your previous lab and complete the sections marked with `TODO X` comments.

In [ ]:
import os
import numpy as np
# TODO 01  - Load the data from LAB 01

save_path =...
X = np.load(...)
X_test = np.load(...)
y = np.load(...)
y_test = np.load(...)



## **Challenge 02**

Now, let’s build our multi-layer perceptronmodel to detect wildfires!

In [ ]:
#TODO 01 - Let’s begin with a simple Multi-Layer Perceptron (MLP) model.
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.optimizers import Adam

input_shape = ...

model = Sequential([
    Input(input_shape),
    Flatten(),
    Dense(...),
    Dense(1, activation='sigmoid')  # Use a single output neuron with a sigmoid activation function for binary classification.
])

# Display the model architecture to visualise its layers and parameters.
model.summary()

In [ ]:
#TODO 02 - Let’s compile our MLP model with an appropriate loss function, optimizer, and evaluation metrics.

model.compile(
    optimizer=...,
    loss=...,  # Use a loss function compatible with binary classification problems.
    metrics=['accuracy']
)



In [ ]:
#TODO 03 - Let’s train our MLP model using the training data while monitoring its performance on the validation set.
import time
start_time = time.time()


history = model.fit(
    ..., ..., # Training Data
    epochs=...,
    batch_size=...,
    validation_split=...,  # Use 20% of the training data as a validation set
    verbose=1,
)



end_time = time.time()
training_time = end_time - start_time

print("Training Time (s): ", training_time)

In [ ]:
#TODO 04 - Evaluate the MLP model on the test data to assess its performance.
model.evaluate(..., ...) # Evaluate the model using data that it has never seen before to ensure an unbiased assessment of its generalisation ability.


Let’s visualise the training and validation curves to assess the model's performance. Analyse the curves to determine if there is overfitting or underfitting.


In [ ]:
import matplotlib.pyplot as plt

def Plot_acc_loss(history):
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']

  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs_range = range(len(loss))

  plt.figure(figsize=(8, 8))
  plt.subplot(2, 1, 1)
  plt.plot(epochs_range, acc, label='Training Accuracy')
  plt.plot(epochs_range, val_acc, label='Validation Accuracy')
  plt.legend(loc='lower right')
  plt.title('Training and Validation Accuracy')

  plt.subplot(2, 1, 2)
  plt.plot(epochs_range, loss, label='Training Loss')
  plt.plot(epochs_range, val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.title('Training and Validation Loss')
  plt.show()
Plot_acc_loss(history)

Now, let’s evaluate our model using classification metrics such as accuracy, precision, recall, and F1-score to gain deeper insights into its performance.

In [ ]:
from sklearn.metrics import classification_report
pred = model.predict(X_test)
binary_predictions = [1 if p > 0.5 else 0 for p in pred]


print(classification_report(y_test, binary_predictions))

And let’s analyse the confusion matrix to understand the model's performance in correctly and incorrectly classifying each class.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import pandas as pd
#TODO 05 - Complete the function to calculate and display the classification metrics and the confusion matrix for the model's predictions.


def plot_confusion_matrix(y_true, y_pred):
  cm = confusion_matrix(..., ...)

  # Define class labels
  class_labels = [..., ...]

  # Create a DataFrame
  cm_df = pd.DataFrame(cm, index=class_labels, columns=class_labels)

  plt.figure(figsize=(8,6))
  sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')
  plt.ylabel('Actual')
  plt.xlabel('Predicted')
  plt.title('Confusion Matrix')
  plt.show()
plot_confusion_matrix(..., ...)



Finally, let’s visualise the images along with their predicted and true labels to better understand the model’s performance.

In [ ]:
import random
reverse_label_map = {
    0:"Fire",1:"Non_Fire"
}
def view_grid_figures(predictions):
  combined_list = list(zip(X_test, predictions))
  Samples = random.sample(combined_list, 16)

  total_images = 16
  grid_size = (4,4)

  fig, axes = plt.subplots(grid_size[0], grid_size[1])
  axes = axes.flatten()

  for idx in range(total_images):
      ax = axes[idx]
      ax.axis('off')


      img = Samples[idx][0]
      label = Samples[idx][1]
      ax.imshow(img)

      ax.set_title(reverse_label_map[label], fontsize=10)

  plt.tight_layout()
  plt.show()

view_grid_figures(binary_predictions)

Analysing Model Inference Time

In [ ]:
inference_times = []

# Perform inference and record times
for i in range(len(X_test)):
    start_time = time.time()
    _ = model.predict(np.array([X_test[i,:,:,:]]), verbose=0)
    end_time = time.time()
    inference_times.append(end_time - start_time)

# Calculate average and standard deviation
avg_time = np.mean(inference_times)
std_time = np.std(inference_times)

print(f"Training Time: {training_time} s")
print(f'Average Inference Time over {len(X_test)} runs: {avg_time*1000:.2f} ms')
print(f'Standard Deviation: {std_time*1000:.2f} ms')